# Processus Gaussien pour prédiction poids

### Librairies

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, RBF

### Données brutes

In [ ]:
file_save = os.getcwd() + "/data/dataset.csv"

with open(file_save, 'r') as f:
    df = pd.read_csv(file_save, index_col=0)

In [ ]:
df

In [ ]:
# clean-up
df1 = df.copy(deep=True)
columns_to_drop = ['Masse_Osseuse', 'Masse_Musculaire', 'Masse_Hydrique', 'MG%', 'BMR', 'Lipides', 'Glucides', 'Proteines', 'exo_duree', 'exo_cals_bruts', 'Depense_cal_totale', 'cal_deficit']
df1.drop(columns=columns_to_drop, inplace=True)

In [ ]:
df1

In [ ]:
# Visu

fig, axs = plt.subplots(2,2,figsize=(24,8))

for i,name in enumerate(df1.columns):
    r = i%2
    c = i//2
    axs[r,c].set_title(name)
    df1[name].plot(ax=axs[r,c])
    axs[r,c].grid(True)

In [ ]:
# need to check that datapoints follow by one day
# and deal with holes

## Création du dataset

### Step 1 : moyennage des données

In [ ]:
length_average = 3

In [ ]:
def get_df_moyenne(df, length_average=length_average):
    """prend la dataframe et moyenne les valeurs suivant length_average"""
    df = df.rolling(window=length_average).mean()
    df.dropna(inplace=True)
    return df

In [ ]:
df2 = get_df_moyenne(df1)

df2

### Step 2 : create targets, which are next days's Masse_Totale and Masse_Grasse values

In [ ]:
def create_targets(df=df2):
    """créé les targets
    """
    df2 = pd.concat([df.shift(1), df['Masse_Totale'], df[]'Masse_Grasse']], axis=1)
    df2.columns = ['MT', 'MG', 'Cals', 'Exos', 'MT+1', 'MG+1']
    df2.dropna(inplace=True)
    
    return df2

In [ ]:

df2 = create_targets(df2)

### Step 3 : date de démarrage des données d'entraînement

In [ ]:
year = 2021
month = 5
day = 1

In [ ]:
from datetime import datetime

def crop_dataset(df=df2, year=year, month=month, day=day):
    """supprime les données avant year,month, date
    """
    df.index = pd.to_datetime(df.index)
    df = df[df.index >= datetime(year=year, month=month, day=day)]
    return df

In [ ]:
df2 = crop_dataset().copy(deep=True)

df2

### Step 4 : Construit les train et test sets

In [ ]:
# longueur du test set
LAST = 90

In [ ]:
def create_train_test_sets(df=df2, last=LAST):
    """create train and test datasets"""
    X = df2[['MT', 'MG', 'Cals', 'Exos']]
    y_mt = df2['MT+1']
    y_mg = df2['MG+1']
    
    X_train = X[:-LAST]
    X_test = X[-LAST:]

    y_mt_train = y_mt[:-LAST]
    y_mt_test = y_mt[-LAST:]

    y_mg_train = y_mg[:-LAST]
    y_mg_test = y_mg[-LAST:]
    
    return X_train, X_test, y_mt_train, y_mt_test, y_mg_train, y_mg_test

In [ ]:
X_train, X_test, y_mt_train, y_mt_test, y_mg_train, y_mg_test = create_train_test_sets()

### Instancie un GP par output

In [ ]:
# Gaussian process

def get_gpr(length_scale=7, length_scale_bounds=(1e-1, 1e4), noise_level=2.0, noise_level_bounds=(1e-6, 1e2)):
    """Instancie un GPR avec un kernel Gaussien + kernel bruit blanc"""

    kernel = RBF(
        length_scale=length_scale,
        length_scale_bounds=length_scale_bounds
        ) \
        + WhiteKernel(
            noise_level=noise_level,
            noise_level_bounds=noise_level_bounds
            )
    gpr = GaussianProcessRegressor(kernel = kernel, alpha=0.0, random_state=42, normalize_y=True, n_restarts_optimizer=9 )
    
    return gpr, kernel

In [ ]:
gpr_mt, kernel_mt = get_gpr()
gpr_mg, kernel_mg = get_gpr()

### Training

In [ ]:
gpr_mt.fit(X_train, y_mt_train)

In [ ]:
gpr_mg.fit(X_train, y_mg_train)

In [ ]:
mean_prediction_mt, std_prediction_mt = gpr_mt.predict(X_train, return_std=True)

In [ ]:
mean_prediction_mg, std_prediction_mg = gpr_mg.predict(X_train, return_std=True)

In [ ]:
def plot_training(X_train, y_train, y_mean, y_std, gpr, kernel, titre):
    """utility function pour afficher perf sur training set"""
    
    fig, ax = plt.subplots(figsize=(10,6))

    abscisses = np.arange(X_train.shape[0])
    ax.scatter(abscisses, y_train, marker='o', label="Observations", color='blue')
    ax.plot(abscisses, y_mean, marker='.', label="Mean prediction", color='green')
    ax.set_title(
        titre +\
        f" {X_train.shape[0]} points\n" +\
        f"Initial: {kernel}\nOptimum: {gpr.kernel_}\nLog-Marginal-Likelihood: " +\
        f"{gpr.log_marginal_likelihood(gpr.kernel_.theta)}\n" +\
        f"données moyennées sur {length_average} jour(s)"
    )
    ax.grid(True)
    ax.fill_between(
        abscisses,
        y_mean - 1.96 * y_std,
        y_mean + 1.96 * y_std,
        alpha=0.5,
        label=r"95% confidence interval",
    )
    plt.legend()
    
    return fig, ax

In [ ]:
fig, ax = plot_training(X_train, y_mt_train, mean_prediction_mt, std_prediction_mt, gpr_mt, kernel_mt, "Train set MT")

In [ ]:
fig, ax = plot_training(X_train, y_mg_train, mean_prediction_mg, std_prediction_mg, gpr_mg, kernel_mg, "Train set MG")

In [ ]:
# Metrics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error


print(f"Train set : MAE MT = {mean_absolute_error(y_mt_train, mean_prediction_mt)}")
print(f"Train set : MSE MT = {mean_squared_error(y_mt_train, mean_prediction_mt)}")
print(f"Train set : MAE MG = {mean_absolute_error(y_mg_train, mean_prediction_mg)}")
print(f"Train set : MSE MG = {mean_squared_error(y_mg_train, mean_prediction_mg)}")

### Inférences J+1

In [ ]:
def plot_inference(X_test, y_test, y_mean, y_std, gpr, kernel, titre):
    """utility function pour afficher perf sur test set avec CI"""
    
    fig, ax = plt.subplots(figsize=(10,6))

    abscisses = np.arange(X_test.shape[0])
    ax.scatter(abscisses, y_test, marker='o', label="Observations", color='blue')
    ax.plot(abscisses, y_mean, marker='.', label="Mean prediction", color='green')
    ax.set_title(
        titre +\
        f" {X_test.shape[0]} points\n" +\
        f"Initial: {kernel}\nOptimum: {gpr.kernel_}\nLog-Marginal-Likelihood: " +\
        f"{gpr.log_marginal_likelihood(gpr.kernel_.theta)}\n" +\
        f"données moyennées sur {length_average} jour(s)"
    )
    ax.grid(True)
    ax.fill_between(
        abscisses,
        y_mean - 1.96 * y_std,
        y_mean + 1.96 * y_std,
        alpha=0.5,
        label=r"95% confidence interval",
    )
    plt.legend()
    
    return fig, ax

In [ ]:
mean_prediction_mt, std_prediction_mt = gpr_mt.predict(X_test, return_std=True)

In [ ]:
fig, ax = plot_inference(X_test, y_mt_test, mean_prediction_mt, std_prediction_mt, gpr_mt, kernel_mt, "Test set MT J+1")

In [ ]:
mean_prediction_mg, std_prediction_mg = gpr_mg.predict(X_test, return_std=True)

In [ ]:
fig, ax = plot_inference(X_test, y_mg_test, mean_prediction_mg, std_prediction_mg, gpr_mg, kernel_mg, "Test set MG J+1")

In [ ]:
print(f"Test set J+1 : MAE MT = {mean_absolute_error(y_mt_test, mean_prediction_mt)}")
print(f"Test set J+1 : MSE MT = {mean_squared_error(y_mt_test, mean_prediction_mt)}")
print(f"Test set J+1 : MAE MG = {mean_absolute_error(y_mg_test, mean_prediction_mg)}")
print(f"Test set J+1 : MSE MG = {mean_squared_error(y_mg_test, mean_prediction_mg)}")

### Inférence J+1 à J+N

In [ ]:
# horizon de prédiction
horizon = 15

In [ ]:
X_test

In [ ]:
X_test_pred = X_test[:horizon].copy(deep=True)

In [ ]:
y_pred_mt = []  # prédiction MT
y_pred_mg = []  # prédiction MG
y_pred_mt_std = []  # écart-type prédiction MT
y_pred_mg_std = []  # écart-type prédiction MG

# initialisation des valeurs de départ
next_mt = X_test_pred.iloc[0,0]
next_mg = X_test_pred.iloc[0,1]

In [ ]:
for r in range(len(X_test_pred)):
    # on extrait les inputs du jour et on remplace les valeurs MT, MG du jour par celles prédites
    row = X_test_pred.iloc[r:r+1,:].copy(deep=True)
    mini_X_test = pd.DataFrame(
        data = { 'MT' : next_mt,
                'MG' : next_mg,
                'Cals' : row['Cals'],
                'Exos' : row['Exos']}
    )
    # on prédit MT à J+1
    pred_mt, pred_mt_std = gpr_mt.predict(mini_X_test, return_std=True)
    next_mt = pred_mt[0]
    y_pred_mt.append(next_mt)
    y_pred_mt_std.append(pred_mt_std[0])
    # on prédit MG à J+1
    pred_mg, pred_mg_std = gpr_mg.predict(mini_X_test, return_std=True)
    next_mg = pred_mg[0]
    y_pred_mg.append(next_mg)
    y_pred_mg_std.append(pred_mg_std[0])

In [ ]:
y_pred_mt_std = np.array(y_pred_mt_std)
y_pred_mg_std = np.array(y_pred_mg_std)

In [ ]:
def plot_inference_horizon(X_test, y_test, y_mean, y_std, gpr, kernel, titre):
    """utility function pour afficher perf sur test set avec CI"""
    
    fig, ax = plt.subplots(figsize=(10,6))

    abscisses = np.arange(X_test.shape[0])
    ax.scatter(abscisses, y_test[:horizon], marker='o', label="Observations", color='blue')
    ax.plot(abscisses, y_mean, marker='.', label="Mean prediction", color='green')
    ax.set_title(f"Test set {X_test.shape[0]} points. Inférence J+2 avec valeur prédite à J+1\n" f"Initial: {kernel}\nOptimum: {gpr.kernel_}\nLog-Marginal-Likelihood: "
        f"{gpr_mt.log_marginal_likelihood(gpr.kernel_.theta)}\n" \
            f"données moyennées sur {length_average} jours" 
    )
    ax.grid(True)
    ax.fill_between(
        abscisses,
        y_mean - 1.96 * y_std,
        y_mean + 1.96 * y_std,
        alpha=0.5,
        label=r"95% confidence interval",
    )
    plt.legend()
    
    return fig, ax

In [ ]:
fig, ax = plot_inference_horizon(X_test_pred, y_mt_test, y_pred_mt, y_pred_mt_std, gpr_mt, kernel_mt, "Prediction longue MT")

In [ ]:
fig, ax = plot_inference_horizon(X_test_pred, y_mg_test, y_pred_mg, y_pred_mg_std, gpr_mg, kernel_mg, "Prediction longue MG")

In [ ]:
print(f"Test set J+{horizon} : MAE MT = {mean_absolute_error(y_mt_test[:horizon], y_pred_mt)}")
print(f"Test set J+{horizon} : MSE MT = {mean_squared_error(y_mt_test[:horizon], y_pred_mt)}")
print(f"Test set J+{horizon} : MAE MG = {mean_absolute_error(y_mg_test[:horizon], y_pred_mg)}")
print(f"Test set J+{horizon} : MSE MG = {mean_squared_error(y_mg_test[:horizon], y_pred_mg)}")

## Grid Search

In [ ]:
# hyperparamètres

length_average_list = [3] # durée moyennage des données
start_date_list = [ (2020, 9, 1)]  # démarrage des données

In [ ]:
import itertools

In [ ]:
for length_average, start_date in itertools.product(length_average_list, start_date_list):
    
    year, month, day = start_date
    
    # create dataset
    df2 = get_df_moyenne(df1, length_average=length_average)  # moyenne les données
    df2 = create_targets(df2) # retire les données non-utilisées
    df2 = crop_dataset(year=year, month=month, day=day).copy(deep=True)
    X_train, X_test, y_mt_train, y_mt_test, y_mg_train, y_mg_test = create_train_test_sets()
    
    # create and train models
    gpr_mt, kernel_mt = get_gpr()
    gpr_mt.fit(X_train, y_mt_train)
    gpr_mg, kernel_mg = get_gpr()  
    gpr_mg.fit(X_train, y_mg_train)
    print(f"Log marginal likelihood MT : {gpr_mt.log_marginal_likelihood(gpr_mt.kernel_.theta)}")
    print(f"Log marginal likelihood MG : {gpr_mg.log_marginal_likelihood(gpr_mg.kernel_.theta)}")
    
    # performance metrics sur training set
    mean_prediction_mt, std_prediction_mt = gpr_mt.predict(X_train, return_std=True)
    mean_prediction_mg, std_prediction_mg = gpr_mg.predict(X_train, return_std=True) 
    print(f"Train set : MAE MT = {mean_absolute_error(y_mt_train, mean_prediction_mt)}")
    print(f"Train set : MSE MT = {mean_squared_error(y_mt_train, mean_prediction_mt)}")
    print(f"Train set : MAE MG = {mean_absolute_error(y_mg_train, mean_prediction_mg)}")
    print(f"Train set : MSE MG = {mean_squared_error(y_mg_train, mean_prediction_mg)}")
    
    # inférence J+1, performance
    